<a href="https://colab.research.google.com/github/athens-igg/dfu-models/blob/main/mobilenet_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install tensorflow
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()   # select kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp "kaggle (13).json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d purushomohan/dfu-wagners-classification

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [ ]:


!unzip -q dfu-wagners-classification.zip -d /content/drive

In [ ]:

train_data="/content/drive/Dataset/Training"
valid_data="/content/drive/Dataset/Validation"

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 100
N_CLASSES = 5   # Wagner 0–4 (adjust if you have normal + ulcer separately)

# ============================
# 1) Data Preprocessing & Augmentation
# ============================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_data,  # path to training dataset
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_gen = val_datagen.flow_from_directory(
    valid_data,  # path to validation dataset
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)
# ============================
# 2) Define MobileNetV2 Model
# ============================
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model for transfer learning
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
output = Dense(N_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

# ============================
# 3) Compile
# ============================
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# ============================
# 4) Callbacks
# ============================
checkpoint = ModelCheckpoint(
    "/content/drive/Dataset/best_model.h5",
    monitor="val_loss",   # or "val_loss" if you prefer
    save_best_only=True,
    mode="min"
)

callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, verbose=1),
    checkpoint
]


# ============================
# 5) Train
# ============================
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks
)

# ============================
# 6) Fine-tune (optional)
# ============================
# Unfreeze some top layers
base_model.trainable = True
for layer in base_model.layers[:-30]:  # keep earlier layers frozen
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_gen,
    epochs=50,
    validation_data=val_gen,
    callbacks=callbacks
)


In [ ]:
from tensorflow.keras.models import load_model

# Load the best model saved by ModelCheckpoint
best_model = load_model("/content/drive/Dataset/best_model.h5")

# Evaluate on validation data
val_loss, val_acc = best_model.evaluate(val_gen)
print(f"Best Validation Accuracy: {val_acc*100:.2f}%")
print(f"Best Validation Loss: {val_loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Merge history and fine-tuning history
acc = history.history['accuracy'] + history_ft.history['accuracy']
val_acc = history.history['val_accuracy'] + history_ft.history['val_accuracy']
loss = history.history['loss'] + history_ft.history['loss']
val_loss = history.history['val_loss'] + history_ft.history['val_loss']

epochs_range = range(1, len(acc) + 1)

# Plot accuracy
plt.figure(figsize=(8,6))
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy (with Fine-tuning)')
plt.legend()
plt.show()

# Plot loss
plt.figure(figsize=(8,6))
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss (with Fine-tuning)')
plt.legend()
plt.show()

In [ ]:
model.save('/content/drive/Dataset/best_model.h5')

In [ ]:
from google.colab import files
files.download('/content/drive/Dataset/best_model.h5')

In [ ]:
!pip install opencv-python-headless
from google.colab import files
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Upload image
uploaded = files.upload()
class_names = ["Grade 0", "Grade1", "Grade2", "Grade3", "Normal" ]
filename = list(uploaded.keys())[0]

true_class_name = "Grade 0"  # change based on the image you uploaded
true_class_id = class_names.index(true_class_name)


# Load and preprocess image
IMG_SIZE = (224, 224)
img = cv2.imread(filename)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img_rgb, IMG_SIZE)

# Normalize same as training (rescale 1/255)
img_array = img_resized.astype("float32") / 255.0
img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
# Predict
pred = best_model.predict(img_array)
class_id = np.argmax(pred)
confidence = np.max(pred)

# Show result
plt.imshow(img_rgb)
plt.axis("off")
plt.title(f"Prediction: {class_names[class_id]} ({confidence*100:.2f}%)")
plt.show()

if class_id == true_class_id:
    print("Correctly classified")
else:
    print("Misclassified")



In [ ]:
# Step 1: Upload the image
from google.colab import files
uploaded = files.upload()  # This will open a file picker

# Get the uploaded file path
img_path = list(uploaded.keys())[0]

# Step 2: Load and preprocess the uploaded image
import cv2
import numpy as np

# Read the image
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Resize to your model's expected input size (replace 224 with your size)
img_resized = cv2.resize(img, (224, 224))

# Normalize if your model was trained with normalized images
input_array = np.expand_dims(img_resized / 255.0, axis=0)

# input_array is now ready for Grad-CAM


In [ ]:
# Step 1: Load model
from tensorflow import keras
model = keras.models.load_model('/content/drive/Dataset/best_model.h5')

# Step 2: Load and preprocess uploaded image
import cv2
import numpy as np
#img_path = "Screenshot 2025-12-28 170002 (1).png"  # your uploaded file
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (224, 224))  # replace 224 with your model input size
input_array = np.expand_dims(img_resized / 255.0, axis=0)  # normalize if needed

# Step 3: Identify last convolutional layer
last_conv_layer_name = None
for layer in model.layers[::-1]:
    if 'conv' in layer.name:
        last_conv_layer_name = layer.name
        break
print("Last convolutional layer:", last_conv_layer_name)

# Step 4: Grad-CAM heatmap generation
import tensorflow as tf

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize heatmap from 0 to 1 for color mapping
    heatmap = heatmap - tf.reduce_min(heatmap)
    heatmap = heatmap / tf.reduce_max(heatmap)
    return heatmap.numpy()

heatmap = make_gradcam_heatmap(input_array, model, last_conv_layer_name)

# Step 5: Overlay heatmap on original image with clear blue → red gradient
heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap_resized = np.uint8(255 * heatmap_resized)
heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)  # Jet: blue=low, red=high
superimposed_img = cv2.addWeighted(img, 0.6, heatmap_color, 0.4, 0)

# Step 6: Display Grad-CAM
import matplotlib.pyplot as plt
plt.figure(figsize=(8,8))
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()

# Optional: save the Grad-CAM image
cv2.imwrite("gradcam_output.png", cv2.cvtColor(superimposed_img, cv2.COLOR_RGB2BGR))
print("Grad-CAM saved as gradcam_output.png")



In [ ]:
# Step 1: Install LIME
!pip install lime

# Step 2: Load model and image
from tensorflow import keras
import cv2
import numpy as np

model = keras.models.load_model('/content/drive/Dataset/best_model.h5')

# Load uploaded image

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (224, 224))
input_array = np.expand_dims(img_resized / 255.0, axis=0)

# Step 3: LIME setup
from lime import lime_image
from skimage.segmentation import slic

explainer = lime_image.LimeImageExplainer()

# Define prediction function for LIME
def predict_fn(images):
    images = np.array(images) / 255.0  # normalize
    return model.predict(images)

# Step 4: Generate explanation
explanation = explainer.explain_instance(
    img_resized,                   # original image
    predict_fn,                    # prediction function
    top_labels=1,                  # explain top predicted class
    hide_color=0,
    num_samples=1000               # increase for smoother explanation
)

# Step 5: Get image and mask for top class
from skimage.color import label2rgb

top_label = explanation.top_labels[0]
temp, mask = explanation.get_image_and_mask(
    top_label,
    positive_only=False,
    num_features=10,
    hide_rest=False
)
img_boundry = label2rgb(mask, temp, bg_label=0)

# Step 6: Display and save explanation
import matplotlib.pyplot as plt
plt.figure(figsize=(6,6))
plt.imshow(img_boundry)
plt.axis('off')
plt.show()

# Save the LIME overlay
from matplotlib import image as mpimg
mpimg.imsave("lime_output.png", img_boundry)
print("LIME overlay saved as lime_output.png")


In [ ]:
!pip install opencv-python-headless
from google.colab import files
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Load the model
model = keras.models.load_model('/content/drive/Dataset/best_model.h5')
# Upload image
uploaded = files.upload()
class_names = ["Grade 0", "Grade 1", "Grade 2", "Grade 3", "Normal" ]
filename = list(uploaded.keys())[0]

true_class_name = "Grade 2"  # change based on the image you uploaded
true_class_id = class_names.index(true_class_name)


# Load and preprocess image
IMG_SIZE = (224, 224)
img = cv2.imread(filename)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img_rgb, IMG_SIZE)

# Normalize same as training (rescale 1/255)
img_array = img_resized.astype("float32") / 255.0
img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
# Predict
pred = model.predict(img_array)
class_id = np.argmax(pred)
confidence = np.max(pred)

# Show result
plt.imshow(img_rgb)
plt.axis("off")
plt.title(f"Prediction: {class_names[class_id]} ({confidence*100:.2f}%)")
plt.show()

if class_id == true_class_id:
    print("Correctly classified")
else:
    print("Misclassified")



In [ ]:
# Step 1: Upload the image
from google.colab import files
uploaded = files.upload()  # This will open a file picker

# Get the uploaded file path
img_path = list(uploaded.keys())[0]

# Step 2: Load and preprocess the uploaded image
import cv2
import numpy as np

# Read the image
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Resize to your model's expected input size (replace 224 with your size)
img_resized = cv2.resize(img, (224, 224))

# Normalize if your model was trained with normalized images
input_array = np.expand_dims(img_resized / 255.0, axis=0)

# input_array is now ready for Grad-CAM


In [ ]:
# Step 1: Load model
from tensorflow import keras
model = keras.models.load_model('/content/drive/Dataset/best_model.h5')

# Step 2: Load and preprocess uploaded image
import cv2
import numpy as np
#img_path = "Screenshot 2025-12-28 170002 (1).png"  # your uploaded file
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (224, 224))  # replace 224 with your model input size
input_array = np.expand_dims(img_resized / 255.0, axis=0)  # normalize if needed

# Step 3: Identify last convolutional layer
last_conv_layer_name = None
for layer in model.layers[::-1]:
    if 'conv' in layer.name:
        last_conv_layer_name = layer.name
        break
print("Last convolutional layer:", last_conv_layer_name)

# Step 4: Grad-CAM heatmap generation
import tensorflow as tf

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize heatmap from 0 to 1 for color mapping
    heatmap = heatmap - tf.reduce_min(heatmap)
    heatmap = heatmap / tf.reduce_max(heatmap)
    return heatmap.numpy()

heatmap = make_gradcam_heatmap(input_array, model, last_conv_layer_name)

# Step 5: Overlay heatmap on original image with clear blue → red gradient
heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap_resized = np.uint8(255 * heatmap_resized)
heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)  # Jet: blue=low, red=high
superimposed_img = cv2.addWeighted(img, 0.6, heatmap_color, 0.4, 0)

# Step 6: Display Grad-CAM
import matplotlib.pyplot as plt
plt.figure(figsize=(8,8))
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()

# Optional: save the Grad-CAM image
cv2.imwrite("gradcam_output.png", cv2.cvtColor(superimposed_img, cv2.COLOR_RGB2BGR))
print("Grad-CAM saved as gradcam_output.png")



In [ ]:
# Step 1: Install LIME
!pip install lime

# Step 2: Load model and image
from tensorflow import keras
import cv2
import numpy as np

model = keras.models.load_model('/content/drive/Dataset/best_model.h5')

# Load uploaded image

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (224, 224))
input_array = np.expand_dims(img_resized / 255.0, axis=0)

# Step 3: LIME setup
from lime import lime_image
from skimage.segmentation import slic

explainer = lime_image.LimeImageExplainer()

# Define prediction function for LIME
def predict_fn(images):
    images = np.array(images) / 255.0  # normalize
    return model.predict(images)

# Step 4: Generate explanation
explanation = explainer.explain_instance(
    img_resized,                   # original image
    predict_fn,                    # prediction function
    top_labels=1,                  # explain top predicted class
    hide_color=0,
    num_samples=1000               # increase for smoother explanation
)

# Step 5: Get image and mask for top class
from skimage.color import label2rgb

top_label = explanation.top_labels[0]
temp, mask = explanation.get_image_and_mask(
    top_label,
    positive_only=False,
    num_features=10,
    hide_rest=False
)
img_boundry = label2rgb(mask, temp, bg_label=0)

# Step 6: Display and save explanation
import matplotlib.pyplot as plt
plt.figure(figsize=(6,6))
plt.imshow(img_boundry)
plt.axis('off')
plt.show()

# Save the LIME overlay
from matplotlib import image as mpimg
mpimg.imsave("lime_output.png", img_boundry)
print("LIME overlay saved as lime_output.png")


In [ ]:
!pip install opencv-python-headless
from google.colab import files
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Load the model
model = keras.models.load_model('/content/drive/Dataset/best_model.h5')
# Upload image
uploaded = files.upload()
class_names = ["Grade 0", "Grade 1", "Grade 2", "Grade 3", "Normal" ]
filename = list(uploaded.keys())[0]

true_class_name = "Grade 3"  # change based on the image you uploaded
true_class_id = class_names.index(true_class_name)


# Load and preprocess image
IMG_SIZE = (224, 224)
img = cv2.imread(filename)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img_rgb, IMG_SIZE)

# Normalize same as training (rescale 1/255)
img_array = img_resized.astype("float32") / 255.0
img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
# Predict
pred = model.predict(img_array)
class_id = np.argmax(pred)
confidence = np.max(pred)

# Show result
plt.imshow(img_rgb)
plt.axis("off")
plt.title(f"Prediction: {class_names[class_id]} ({confidence*100:.2f}%)")
plt.show()

if class_id == true_class_id:
    print("Correctly classified")
else:
    print("Misclassified")



In [ ]:
# Step 1: Upload the image
from google.colab import files
uploaded = files.upload()  # This will open a file picker

# Get the uploaded file path
img_path = list(uploaded.keys())[0]

# Step 2: Load and preprocess the uploaded image
import cv2
import numpy as np

# Read the image
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Resize to your model's expected input size (replace 224 with your size)
img_resized = cv2.resize(img, (224, 224))

# Normalize if your model was trained with normalized images
input_array = np.expand_dims(img_resized / 255.0, axis=0)

# input_array is now ready for Grad-CAM


In [ ]:
# Step 1: Load model
from tensorflow import keras
model = keras.models.load_model('/content/drive/Dataset/best_model.h5')

# Step 2: Load and preprocess uploaded image
import cv2
import numpy as np
#img_path = "Screenshot 2025-12-28 170002 (1).png"  # your uploaded file
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (224, 224))  # replace 224 with your model input size
input_array = np.expand_dims(img_resized / 255.0, axis=0)  # normalize if needed

# Step 3: Identify last convolutional layer
last_conv_layer_name = None
for layer in model.layers[::-1]:
    if 'conv' in layer.name:
        last_conv_layer_name = layer.name
        break
print("Last convolutional layer:", last_conv_layer_name)

# Step 4: Grad-CAM heatmap generation
import tensorflow as tf

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize heatmap from 0 to 1 for color mapping
    heatmap = heatmap - tf.reduce_min(heatmap)
    heatmap = heatmap / tf.reduce_max(heatmap)
    return heatmap.numpy()

heatmap = make_gradcam_heatmap(input_array, model, last_conv_layer_name)

# Step 5: Overlay heatmap on original image with clear blue → red gradient
heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap_resized = np.uint8(255 * heatmap_resized)
heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)  # Jet: blue=low, red=high
superimposed_img = cv2.addWeighted(img, 0.6, heatmap_color, 0.4, 0)

# Step 6: Display Grad-CAM
import matplotlib.pyplot as plt
plt.figure(figsize=(8,8))
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()

# Optional: save the Grad-CAM image
cv2.imwrite("gradcam_output.png", cv2.cvtColor(superimposed_img, cv2.COLOR_RGB2BGR))
print("Grad-CAM saved as gradcam_output.png")



In [ ]:
# Step 1: Load your model
from tensorflow import keras
model = keras.models.load_model('/content/drive/Dataset/best_model.h5')

# Step 2: Prepare the uploaded image
import cv2
import numpy as np
#img_path = "Screenshot 2025-12-28 170002 (1).png"  # your uploaded file
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (224, 224))  # replace 224 with your model input size
input_array = np.expand_dims(img_resized / 255.0, axis=0)  # normalize if needed

# Step 3: Identify last conv layer
last_conv_layer_name = None
for layer in model.layers[::-1]:
    if 'conv' in layer.name:
        last_conv_layer_name = layer.name
        break
print("Last convolutional layer:", last_conv_layer_name)

# Step 4: Generate Grad-CAM heatmap
import tensorflow as tf

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

heatmap = make_gradcam_heatmap(input_array, model, last_conv_layer_name)

# Step 5: Overlay heatmap on original image
heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap_resized = np.uint8(255 * heatmap_resized)
heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)
superimposed_img = cv2.addWeighted(img, 0.6, heatmap_color, 0.4, 0)

# Step 6: Display Grad-CAM
import matplotlib.pyplot as plt
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()


In [ ]:
# Step 1: Install LIME
!pip install lime

# Step 2: Load model and image
from tensorflow import keras
import cv2
import numpy as np

model = keras.models.load_model('/content/drive/Dataset/best_model.h5')

# Load uploaded image

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (224, 224))
input_array = np.expand_dims(img_resized / 255.0, axis=0)

# Step 3: LIME setup
from lime import lime_image
from skimage.segmentation import slic

explainer = lime_image.LimeImageExplainer()

# Define prediction function for LIME
def predict_fn(images):
    images = np.array(images) / 255.0  # normalize
    return model.predict(images)

# Step 4: Generate explanation
explanation = explainer.explain_instance(
    img_resized,                   # original image
    predict_fn,                    # prediction function
    top_labels=1,                  # explain top predicted class
    hide_color=0,
    num_samples=1000               # increase for smoother explanation
)

# Step 5: Get image and mask for top class
from skimage.color import label2rgb

top_label = explanation.top_labels[0]
temp, mask = explanation.get_image_and_mask(
    top_label,
    positive_only=False,
    num_features=10,
    hide_rest=False
)
img_boundry = label2rgb(mask, temp, bg_label=0)

# Step 6: Display and save explanation
import matplotlib.pyplot as plt
plt.figure(figsize=(6,6))
plt.imshow(img_boundry)
plt.axis('off')
plt.show()

# Save the LIME overlay
from matplotlib import image as mpimg
mpimg.imsave("lime_output.png", img_boundry)
print("LIME overlay saved as lime_output.png")
